<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Ada Tech [DS-PY-004] Técnicas de Programação I (PY) Aulas 4 e 5 : Pandas - Tidy Data.

## Intro

Dizemos que um conjunto de dados é ordenado quando:
* Cada variável é uma coluna
* Cada observação é uma linha
* Cada tipo de unidade observacional forma uma tabela

Algumas definições:
* Variável: É a medida de um atributo, por exemplo, peso, altura, etc.
* Valor: É a medida que uma variável leva para uma observação.
* Observação: Todas as observações assumem o mesmo tipo de valores para cada variável.

## Dataset

Usaremos o conjunto de dados das obras do [Met](https://github.com/metmuseum/openaccess/) (Museu Metropolitano de Arte) que vimos na aula de Limpeza de Dados.

Analisando o conjunto de dados do Met vemos que os dados dos artistas estão na mesma tabela que os dados das obras, quebrando a terceira regra que diz “Cada tipo de unidade observacional forma uma tabela”.

Vamos ver como resolver esse ponto.

## Imports

In [1]:
import pandas as pd
import numpy as np

## Importamos os dados

Nós vemos o quão grande é o conjunto de dados e imprimimos os primeiros registros.

In [2]:
# local
data_location = "../Data/MetObjects_sample.csv"

data = pd.read_csv(data_location)

data.dtypes

Unnamed: 0                   int64
Object Number               object
Is Highlight                  bool
Is Public Domain              bool
Is Timeline Work              bool
Object ID                    int64
Department                  object
AccessionYear               object
Object Name                 object
Title                       object
Culture                     object
Period                      object
Dynasty                     object
Reign                       object
Portfolio                   object
Artist Role                 object
Artist Prefix               object
Artist Display Name         object
Artist Display Bio          object
Artist Suffix               object
Artist Alpha Sort           object
Artist Nationality          object
Artist Begin Date           object
Artist End Date             object
Artist Gender               object
Artist ULAN URL             object
Artist Wikidata URL         object
Object Date                 object
Object Begin Date   

In [3]:
data.head(3)

,Unnamed: 0,Object Number,Is Highlight,Is Public Domain,Is Timeline Work,Object ID,Department,AccessionYear,Object Name,Title,...,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL
0,297513,Inst.1980.3.1,False,False,False,442985,Islamic Art,NaN,Illustrated single work,Illustrated Single Work,...,NaN,NaN,Codices,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Abstraction,http://vocab.getty.edu/page/aat/300056508
1,366196,175T49 R43,True,False,False,591826,The Libraries,NaN,NaN,Vita del Tintoretto,...,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q29385980,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
2,380983,WW.550,False,False,False,650203,Drawings and Prints,NaN,Print,Flowers and Dragonfly,...,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Dragonflies|Flowers,http://vocab.getty.edu/page/aat/300310476|http...


## Tidy data

Analisando o conjunto de dados do Met vemos que os dados dos artistas estão na mesma tabela que os dados das obras, quebrando a terceira regra que diz “Cada tipo de unidade observacional forma uma tabela”.

Neste caso devemos ter uma tabela para Obra e outra para Artista.

Vamos separar esses dados em duas tabelas e eliminar duplicatas na tabela de artistas.

In [4]:
data_artist_columns = ['Artist Role', 'Artist Prefix', 'Artist Display Name',
                       'Artist Display Bio', 'Artist Suffix', 'Artist Alpha Sort',
                       'Artist Nationality', 'Artist Begin Date', 'Artist End Date',
                       'Artist Gender', 'Artist ULAN URL', 'Artist Wikidata URL'
                      ]

data_artist  = data.loc[ : , data_artist_columns]
print(data_artist.shape)

(4743, 12)


In [5]:
data_artist.head(3)

,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL
0,Artist,NaN,Mohammad Aslam Kamal,b. 1939,NaN,Kamal Mohammad Aslam,Pakistani,1914,1990,NaN,NaN,NaN
1,Author,NaN,Carlo Ridolfi,"Italian, Lonigo 1594–1658 Venice",NaN,"Ridolfi, Carlo",Italian,1594,1658,NaN,http://vocab.getty.edu/page/ulan/500013305,https://www.wikidata.org/wiki/Q776990
2,Artist|Publisher,NaN,Maria Pranke|Wiener Werkstätte,"Hungarian, Pándorfalu Parndorf, Hungary 1891–1...",NaN,"Pranke, Maria|Wiener Werkstätte",Hungarian,1891,1972,|,|http://vocab.getty.edu/page/ulan/500268774,|https://www.wikidata.org/wiki/Q241338


Vamos ver se há registros duplicados em data_artist.

Consideramos duplicados os registros que correspondem aos campos "Artist Display Name", "Artist Role".

In [6]:
artist_duplicated = data_artist.duplicated(subset = ["Artist Display Name", "Artist Role"])
any(artist_duplicated)

True

Eliminamos registros duplicados:

In [7]:
data_artist_unique = data_artist.drop_duplicates(subset = ["Artist Display Name", "Artist Role"], keep = "first")
print(data_artist_unique.shape)
print(data_artist.shape)

(1615, 12)
(4743, 12)


Para não perder a associação entre a obra e o artista, na tabela data_object mantemos os campos "Artist Display Name" e "Artist Role", que são os identificadores do artista na tabela data_artist.

In [8]:
data_artist_columns_key =  ['Artist Prefix', 'Artist Display Bio', 'Artist Suffix', 'Artist Alpha Sort',
                            'Artist Nationality', 'Artist Begin Date', 'Artist End Date',
                            'Artist Gender', 'Artist ULAN URL', 'Artist Wikidata URL'
                           ]

data_object = data.drop(data_artist_columns_key, axis = 'columns')
print(data_object.shape)
data_object.columns

(4743, 42)


Index(['Unnamed: 0', 'Object Number', 'Is Highlight', 'Is Public Domain',
       'Is Timeline Work', 'Object ID', 'Department', 'AccessionYear',
       'Object Name', 'Title', 'Culture', 'Period', 'Dynasty', 'Reign',
       'Portfolio', 'Artist Role', 'Artist Display Name', 'Object Date',
       'Object Begin Date', 'Object End Date', 'Medium', 'Dimensions',
       'Credit Line', 'Geography Type', 'City', 'State', 'County', 'Country',
       'Region', 'Subregion', 'Locale', 'Locus', 'Excavation', 'River',
       'Classification', 'Rights and Reproduction', 'Link Resource',
       'Object Wikidata URL', 'Metadata Date', 'Repository', 'Tags',
       'Tags AAT URL'],
      dtype='object')

Quando removemos as duplicatas, a tabela de artistas passou de 4743 registros para 1615.

Isso resulta em uma melhora no desempenho em termos de espaço e facilita a manutenção da consistência dos valores nos registros. Como desvantagem, precisaremos combinar as duas tabelas para responder a algumas perguntas que envolvem relacionamentos entre colunas em tabelas diferentes.

## Revisão de joins con pandas

`merge` `concat` `join` `append`

Como exercício, vamos agora fazer o caminho inverso, partindo de duas tabelas (objetos e artistas) vamos combiná-las para obter um único DataFrame com o conjunto de dados total.

Como vimos na Prática Guiada 3, os dados contidos nos objetos pandas podem ser combinados usando os métodos

- [`pandas.DataFrame.merge`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html): combina linhas de dois DataFrames com base em uma ou mais chaves. É análogo ao join do SQL.

- [`pandas.concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html): concatenar ou empilhar objetos pandas em qualquer um dos eixos.

- [`pandas.DataFrame.join`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html): adicionar colunas de outro DataFrame.

- [`pandas.DataFrame.append`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html): adicione as linhas de outro DataFrame ao final.

**Vamos ver um exemplo de `merge`**, vamos comparar seu tamanho com o DataFrame original (antes de dividi-lo em data_objet e data_artist).

In [9]:
data_all_merge = data_object.merge(data_artist_unique, 
                                   left_on = ['Artist Role', 'Artist Display Name'],
                                   right_on = ['Artist Role', 'Artist Display Name'], 
                                   how = "left"
                                  )

In [10]:
data_all_merge.shape

(4743, 52)

In [11]:
data_all_merge.shape

(4743, 52)

Estamos definindo as colunas 'Artist Role', 'Artist Display Name'  como chaves dos registros para combiná-los em ambos os DataFrames.

Duas linhas serão combinadas no DataFrame resultante se corresponderem aos valores das chaves no Conjunto de dados envolvido na fusão.

`left_on` indica quais colunas do primeiro DataFrame (a da esquerda) são a chave na fusão

`right_on` indica quais colunas do segundo DataFrame (a da direita) são a chave na fusão

`how` indica como combinar os objetos
* inner: o resultado tem apenas os registros cujas chaves estão em ambos os DataFrames
* left: o resultado contém todos os registros do primeiro DataFrame (à esquerda), embora a chave possa não estar no segundo DataFrame (à direita)
* rigth: o resultado tem todos os registros do segundo DataFrame (direita), embora a chave possa não estar no primeiro DataFrame (esquerda)



Vamos ver um exemplo de `concat`, axis = 1 indica que concatenamos as colunas dos dois DataFrames. Observe que ele não combina as colunas repetidas, mas, em vez disso, o DataFrame resultante tem uma coluna de cada DataFrame.

In [12]:
data_all_concat = pd.concat([data_object, data_artist_unique],  
                            axis = 1
                           )

print(data_object.shape)
print(data_artist_unique.shape)
print(data_all_concat.shape)

data_all_concat.columns[data_all_concat.columns == 'Artist Display Name']

(4743, 42)
(1615, 12)
(4743, 54)


Index(['Artist Display Name', 'Artist Display Name'], dtype='object')

**Vamos ver um exemplo de `concat`**, axis = 0 que indica que concatenamos as linhas dos dois DataFrames. Observe que a ordem das colunas não precisa ser a mesma em ambos os DataFrames.

In [13]:
data_object_1 = data_object.iloc[0 : 10, : ]

columns_reverse = data_object.columns[ : : -1 ]
data_object_2 = data_object.iloc[10 : 20].loc[ : , columns_reverse]

print(data_object_1.shape)
print(data_object_2.shape)

(10, 42)
(10, 42)


In [14]:
data_object_1_2 = pd.concat([data_object_1, data_object_2],  
                            axis = 0
                           )

print(data_object_1_2.shape)
data_object_1_2.sample(5)

(20, 42)


,Unnamed: 0,Object Number,Is Highlight,Is Public Domain,Is Timeline Work,Object ID,Department,AccessionYear,Object Name,Title,...,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL
1,366196,175T49 R43,True,False,False,591826,The Libraries,NaN,NaN,Vita del Tintoretto,...,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q29385980,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
15,155187,X.21.93,False,False,True,256657,Greek and Roman Art,NaN,Statuette of a ram,Bronze statuette of a ram,...,NaN,NaN,Bronzes,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Rams,http://vocab.getty.edu/page/aat/300250287
0,297513,Inst.1980.3.1,False,False,False,442985,Islamic Art,NaN,Illustrated single work,Illustrated Single Work,...,NaN,NaN,Codices,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Abstraction,http://vocab.getty.edu/page/aat/300056508
4,472119,24.63.2060,False,False,False,838838,Drawings and Prints,NaN,Print,"Victor Hugo, published in Le Masque",...,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
9,297448,x.165.13,False,False,True,442887,Islamic Art,NaN,Coin weight,Coin Weight,...,NaN,NaN,Glass,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Weights and Measures,http://vocab.getty.edu/page/aat/300386648


**Vamos ver um exemplo de `join`**. Este método é muito eficiente (melhor do que merge) porque usa os índices dos DataFrames.

Para isso, vamos definir as colunas  'Artist Role' e  'Artist Display Name' como um índice nos dois objetos.

Anteriormente, devemos eliminar os registros que são nulos em qualquer um dos dois campos dos DataFrames.

Os índices podem ser criados em DataFrames com registros nulos nesses campos, mas o método de junção falha nessas condições.

In [15]:
print("quantidade de números ausentes no campo 'Artist Role' de data_object_index", data_object['Artist Role'].isnull().sum())
print("quantidade de números ausentes no campo 'Artist Role' de data_artist_unique_index", data_artist_unique['Artist Role'].isnull().sum())
print("quantidade de números ausentes no campo 'Artist Display Name' de data_object_index", data_object['Artist Display Name'].isnull().sum())
print("quantidade de números ausentes no campo 'Artist Display Name' de data_artist_unique_index", data_artist_unique['Artist Display Name'].isnull().sum())


quantidade de números ausentes no campo 'Artist Role' de data_object_index 2359
quantidade de números ausentes no campo 'Artist Role' de data_artist_unique_index 14
quantidade de números ausentes no campo 'Artist Display Name' de data_object_index 2345
quantidade de números ausentes no campo 'Artist Display Name' de data_artist_unique_index 1


In [16]:
data_artist_notnull = data_artist_unique.dropna(subset = ['Artist Role', 'Artist Display Name'], 
                                                how = 'any', 
                                                axis = 0
                                               )

data_object_notnull = data_object.dropna(subset = ['Artist Role', 'Artist Display Name'], 
                                         how = 'any', 
                                         axis = 0
                                        )

In [17]:
data_artist_notnull_index = data_artist_notnull.set_index(['Artist Role', 'Artist Display Name'])
data_object_notnull_index = data_object_notnull.set_index(['Artist Role', 'Artist Display Name'])

In [18]:
data_all_join = data_object_notnull_index.join(data_artist_notnull_index, 
                                               lsuffix = "object_", 
                                               rsuffix = "artist_"
                                              )

print(data_all_join.shape)
data_all_join.sample(3)

(2384, 50)


,,Unnamed: 0,Object Number,Is Highlight,Is Public Domain,Is Timeline Work,Object ID,Department,AccessionYear,Object Name,Title,...,Artist Prefix,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL
Artist Role,Artist Display Name,,,,,,,,,,,,,,,,,,,,,
Artist,"Anonymous, British, 19th century",466974,Ref.BellaLandauer.54,False,False,False,824091,Drawings and Prints,NaN,"Print, Trade Card","Trade Card for Reeve, Bookseller",...,NaN,NaN,NaN,"Anonymous, British, 19th century",NaN,1800,1900,NaN,NaN,NaN
Designer,Margarete Willers,325265,1985.198.76,False,False,False,484111,Modern and Contemporary Art,1985,Textile sample,Bauhaus Archive,...,NaN,"German, Oldenburg 1883–1977 Essen",NaN,"Willers, Margarete",German,1883,1977,Female,http://vocab.getty.edu/page/ulan/500125055,NaN
Artist,Adelaide Milton de Groot,16393,67.187.212,False,False,False,19350,The American Wing,1967,Painting,Fern and Peonies,...,NaN,"American, New York 1876–1967 New York",NaN,"de Groot, Adelaide Milton",American,1876,1967,Female,http://vocab.getty.edu/page/ulan/500015606,https://www.wikidata.org/wiki/Q20936579


Para transformar o índice em duas colunas do DataFrame, usamos o método [`pandas.DataFrame.reset_index()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html).

In [19]:
data_all_join = data_all_join.reset_index()
data_all_join.sample(3)

,Artist Role,Artist Display Name,Unnamed: 0,Object Number,Is Highlight,Is Public Domain,Is Timeline Work,Object ID,Department,AccessionYear,...,Artist Prefix,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL
38,Artist,Allart van Everdingen,289734,20.75.2,False,False,True,427740,Drawings and Prints,1920,...,NaN,"Dutch, Alkmaar 1621–1675 Amsterdam",NaN,"Everdingen, Allart van",Dutch,1621,1675,NaN,http://vocab.getty.edu/page/ulan/500115159,NaN
1620,Author|Translator|Publisher|Artist|Artist,Ovid|Abbé Banier|Chez Despilly|Pierre François...,289765,17.3.1731,False,False,False,427788,Drawings and Prints,1917,...,After,"Roman, Sulmo 43 B.C.–A.D. 17 Tomis, Moesia|Fre...",NaN,"Ovid|Banier, Abbé|Chez Despilly|Basan, Pierre ...",Roman|French|French|French|French,-0043 |1700 |1723 |1741,0017 |1900 |1797 |1814,||||,http://vocab.getty.edu/page/ulan/500246979|||h...,||||
2330,Publisher|Lithographer,"W. Duke, Sons & Co.|Knapp & Company",289426,63.350.205.106.84,False,False,False,427233,Drawings and Prints,1963,...,Issued by,"New York and Durham, N.C.|American, New York",NaN,"Duke, W. Sons & Co.|Knapp & Company",American|American,1870,1920,|,|,|


**Vejamos um exemplo de `append`**, que é semelhante a` concat`

In [20]:
data_object_1 = data_object.iloc[0:10, :]

columns_reverse = data_object.columns[ : : -1 ]
data_object_2 = data_object.iloc[10:20].loc[ : , columns_reverse]

print(data_object_1.shape)
print(data_object_2.shape)

(10, 42)
(10, 42)


Usamos [pd.concat()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) para anexar o dataframes.

In [21]:
data_object_1_2 = pd.concat([data_object_1, data_object_2], ignore_index = True)

print(data_object_1_2.shape)
data_object_1_2.sample(5)

(20, 42)


,Unnamed: 0,Object Number,Is Highlight,Is Public Domain,Is Timeline Work,Object ID,Department,AccessionYear,Object Name,Title,...,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL
15,155187,X.21.93,False,False,True,256657,Greek and Roman Art,NaN,Statuette of a ram,Bronze statuette of a ram,...,NaN,NaN,Bronzes,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Rams,http://vocab.getty.edu/page/aat/300250287
11,361913,X.144.1,False,False,False,569774,Egyptian Art,NaN,"Shabti, Amen-meru",Shabti of Amenmeru,...,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
8,5967,Inst.67.15.49,False,False,False,6370,The American Wing,NaN,Plate,Plate,...,NaN,NaN,Ceramics,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
10,411903,Ref.113,False,False,False,708208,Drawings and Prints,NaN,Print,"Plate 7: the story of the Pisan saints, here S...",...,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
5,471850,2019.601.1,False,False,False,838173,The American Wing,NaN,Drawing,Four Flies,...,NaN,NaN,Drawings,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN


## Referências

Python for Data Analysis. Wes McKinney. Cap 8.2

- [Merge, join, concatenate and compare](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)